<b> 1.1 Install/Import packages </b>

First all the packages required for the workflow are installed and imported

In [1]:
# Please uncomment and install any missing packages 
# You can change the current environment first if needed - https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

# !uv pip install scikit-learn
# !uv pip install warnings
# !uv pip install numpy
# !uv pip install matplotlib
# !uv pip install pandas
# !uv pip install seaborn
# !uv pip install itertools
# !uv pip install pprint
# !uv pip install pydash
# !uv pip install xgboost
# !uv pip install lightgbm
# !uv pip install hyperopt
!uv pip install setuptools

Audited 1 package in 3ms


In [2]:
# Import required libraries/packages

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns             
from timeit import default_timer as timer
import os
import random
import csv
import json
import itertools
import pprint
from pydash import at
import gc
import re

#Import sklearn helper metrics and transformations
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score

#Import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb

#import library for hyperparameter optimization
from hyperopt import STATUS_OK
from hyperopt import hp, tpe, Trials, fmin
from hyperopt.pyll.stochastic import sample

---

<b> 1.2 Set working directory and import datasets </b>

The working directory is set to the address where data is located 

In [3]:
# File system manangement

os.chdir("C:\\Users\\SonNT\\Downloads\\Internship2\\home-credit-default-risk") # set the working directory



Import the main data set and show the feature names and their types 

In [4]:
# Load training data from main file

application_train = pd.read_csv("application_train.csv")
application_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Non-Null Count   Dtype  
---   ------                        --------------   -----  
 0    SK_ID_CURR                    307511 non-null  int64  
 1    TARGET                        307511 non-null  int64  
 2    NAME_CONTRACT_TYPE            307511 non-null  object 
 3    CODE_GENDER                   307511 non-null  object 
 4    FLAG_OWN_CAR                  307511 non-null  object 
 5    FLAG_OWN_REALTY               307511 non-null  object 
 6    CNT_CHILDREN                  307511 non-null  int64  
 7    AMT_INCOME_TOTAL              307511 non-null  float64
 8    AMT_CREDIT                    307511 non-null  float64
 9    AMT_ANNUITY                   307499 non-null  float64
 10   AMT_GOODS_PRICE               307233 non-null  float64
 11   NAME_TYPE_SUITE               306219 non-null  object 
 12   NAME_INCOME_TYPE            

For the main training data, there are 307511 total samples (each row a separate loan) with 122 features of types 41 integer, 65 float and 16 object datatypes. 

Out of these, the feature (SK_ID_CURR) serves as the index and TARGET is the response feature to be predicted. 

Then, all the other data files can be imported.

<i> Since, we do not have access to the labeled version of the test dataset, we will not use it, instead we will split part of the training dataset to use as the test data. </i>

In [5]:
# Load other data files

bureau                = pd.read_csv("bureau.csv")
bureau_balance        = pd.read_csv("bureau_balance.csv")
credit_card_balance   = pd.read_csv("credit_card_balance.csv")
installments_payments = pd.read_csv("installments_payments.csv")
pos_cash_balance      = pd.read_csv("POS_CASH_balance.csv")
previous_application  = pd.read_csv("previous_application.csv")

---

<b> 1.3 Check dataset structures </b>

We can see the number of rows and columns in each file

In [6]:
print ("application_train     :",application_train.shape)
print ("bureau                :",bureau.shape)
print ("bureau_balance        :",bureau_balance.shape)
print ("credit_card_balance   :",credit_card_balance.shape)
print ("installments_payments :",installments_payments.shape)
print ("pos_cash_balance      :",pos_cash_balance.shape)
print ("previous_application  :",previous_application.shape)

application_train     : (307511, 122)
bureau                : (1716428, 17)
bureau_balance        : (27299925, 3)
credit_card_balance   : (3840312, 23)
installments_payments : (13605401, 8)
pos_cash_balance      : (10001358, 8)
previous_application  : (1670214, 37)


The first 3 columns of each dataset are displayed for data inspection

In [7]:
display("application_train")
display(application_train.head(3))
display("bureau")
display(bureau.head(3))
display("bureau_balance")
display(bureau_balance.head(3))
display("credit_card_balance")
display(credit_card_balance.head(3))
display("installments_payments")
display(installments_payments.head(3))
display("pos_cash_balance")
display(pos_cash_balance.head(3))
display("previous_application")
display(previous_application.head(3))

'application_train'

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


'bureau'

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN


'bureau_balance'

...

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000


'pos_cash_balance'

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0


'previous_application'

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0


---

<b> 3.1 Save/Load updated datasets </b> 

We create a checkpoint to save the updated data files if needed

In [8]:
## Save the csv files as a checkpoint if necessary (high memory and time intensive)

save_files = False

if save_files == True:
    application_train.to_csv('application_train_updated.csv', index = False)
    bureau.to_csv('bureau_updated.csv', index = False)
    bureau_balance.to_csv('bureau_balance_updated.csv', index = False)
    credit_card_balance.to_csv('credit_card_balance.csv', index = False)
    installments_payments.to_csv('installments_payments_updated.csv', index = False)
    pos_cash_balance.to_csv('pos_cash_balance_updated.csv', index = False)
    previous_application.to_csv('previous_application_updated.csv', index = False)

This is followed by the option of loading the previously saved updated datafiles, in case we are restarting the notebook

In [9]:
# Load csv files from checkpoint if necessary
# Otherwise, copy to new df and clean up memory 

# load_files = False  # Existing variables in memory

# if load_files == False:
#     application_train_new = application_train.copy()
#     bureau_new = bureau.copy()
#     bureau_balance_new = bureau_balance.copy()
#     credit_card_balance_new = credit_card_balance.copy()
#     installments_payments_new = installments_payments.copy()
#     pos_cash_balance_new = pos_cash_balance.copy()
#     previous_application_new = previous_application.copy()
    
#     gc.enable()
#     del application_train, bureau, bureau_balance, credit_card_balance, installments_payments, pos_cash_balance, previous_application
#     gc.collect()
    
# else:
#     application_train_new = pd.read_csv("application_train_updated.csv")
#     bureau_new = pd.read_csv("bureau_updated.csv")            
#     bureau_balance_new  = pd.read_csv("bureau_balance_updated.csv")      
#     credit_card_balance_new  = pd.read_csv("credit_card_balance.csv")
#     installments_payments_new = pd.read_csv("installments_payments_updated.csv")
#     pos_cash_balance_new = pd.read_csv("pos_cash_balance_updated.csv")     
#     previous_application_new  = pd.read_csv("previous_application_updated.csv")

---

<b> 3.2 Basic Feature Engineering (Replacing outliers, Imputing, One-hot encoding, Rescaling data) </b> 

First, a function is created and run to replace the day outliers previously seen across the data

In [10]:
# def replace_day_outliers(df):
#     """Replace 365243 with np.nan in any columns with DAYS"""
#     for col in df.columns:
#         if "DAYS" in col:
#             df[col] = df[col].replace({365243: np.nan})

#     return df

In [11]:
# # Replace all the day outliers
# application_train_new = replace_day_outliers(application_train_new)
# bureau_new = replace_day_outliers(bureau_new )              
# bureau_balance_new  = replace_day_outliers(bureau_balance_new)      
# credit_card_balance_new  = replace_day_outliers(credit_card_balance_new) 
# installments_payments_new = replace_day_outliers(installments_payments_new )
# pos_cash_balance_new = replace_day_outliers(pos_cash_balance_new )   
# previous_application_new  = replace_day_outliers(previous_application_new )

We create a function to remove columns which have missing values greater than 60%

In [12]:
#Function for removing columns with missing values more than 60%

def remove_missing_col(df):
    miss_data = pd.DataFrame((df.isnull().sum())*100/df.shape[0])
    miss_data_col=miss_data[miss_data[0]>60].index
    data_new  = df[[i for i in df.columns if i not in miss_data_col]]
    return data_new

# PHẦN 1: ÁP DỤNG TỪNG HÀM ĐỂ XEM SỰ THAY ĐỔI CỦA FEATURES

Trong phần này, chúng ta sẽ áp dụng từng bước feature engineering và xem sự thay đổi của dữ liệu sau mỗi bước.

## 1.1. Hàm fix_nulls_outliers - Xử lý giá trị null và outliers

In [13]:
def fix_nulls_outliers(data):
    """
    Xử lý giá trị null và outliers trong Application data
    """
    print(f"Shape trước khi xử lý: {data.shape}")
    print(f"Số lượng null values:\n{data.isnull().sum()[data.isnull().sum() > 0].head(10)}\n")
    
    # Tạo copy để không thay đổi data gốc
    df = data.copy()
    
    # Xử lý các cột categorical
    df['NAME_FAMILY_STATUS'].fillna('Data_Not_Available', inplace=True)
    df['NAME_HOUSING_TYPE'].fillna('Data_Not_Available', inplace=True)
    df['FLAG_MOBIL'].fillna('Data_Not_Available', inplace=True)
    df['FLAG_EMP_PHONE'].fillna('Data_Not_Available', inplace=True)
    df['FLAG_CONT_MOBILE'].fillna('Data_Not_Available', inplace=True)
    df['FLAG_EMAIL'].fillna('Data_Not_Available', inplace=True)
    df['OCCUPATION_TYPE'].fillna('Data_Not_Available', inplace=True)
    df['CNT_FAM_MEMBERS'].fillna(df['CNT_FAM_MEMBERS'].mode()[0], inplace=True)
    
    # Thay 'XNA' bằng 'M'
    df.loc[df['CODE_GENDER'] == 'XNA', 'CODE_GENDER'] = 'M'
    
    # Xử lý NAME_TYPE_SUITE
    df['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace=True)
    
    # Thay 'Unknown' bằng 'Married'
    df.loc[df['NAME_FAMILY_STATUS'] == 'Unknown', 'NAME_FAMILY_STATUS'] = 'Married'
    
    # Xử lý các cột numerical
    # Loại bỏ outlier trong DAYS_EMPLOYED (365243 ≈ 1000 năm)
    df.loc[df['DAYS_EMPLOYED'] == 365243, 'DAYS_EMPLOYED'] = np.nan
    df['DAYS_EMPLOYED'].fillna(df['DAYS_EMPLOYED'].median(), inplace=True)
    
    df['AMT_ANNUITY'].fillna(0, inplace=True)
    df['AMT_GOODS_PRICE'].fillna(0, inplace=True)
    df['EXT_SOURCE_1'].fillna(0, inplace=True)
    df['EXT_SOURCE_2'].fillna(0, inplace=True)
    df['EXT_SOURCE_3'].fillna(0, inplace=True)
    
    print(f"Shape sau khi xử lý: {df.shape}")
    print(f"Số lượng null values còn lại: {df.isnull().sum().sum()}")
    
    return df

In [14]:
# Demo hàm fix_nulls_outliers
print("="*80)
print("DEMO: fix_nulls_outliers")
print("="*80)

train_fixed = fix_nulls_outliers(application_train)
# test_fixed = fix_nulls_outliers(test_data)

# So sánh trước và sau
print("\nTrước khi xử lý:")
print(f"DAYS_EMPLOYED có giá trị 365243: {(application_train['DAYS_EMPLOYED'] == 365243).sum()}")
print(f"\nSau khi xử lý:")
print(f"DAYS_EMPLOYED có giá trị 365243: {(train_fixed['DAYS_EMPLOYED'] == 365243).sum()}")

DEMO: fix_nulls_outliers
Shape trước khi xử lý: (307511, 122)
Số lượng null values:
AMT_ANNUITY            12
AMT_GOODS_PRICE       278
NAME_TYPE_SUITE      1292
OWN_CAR_AGE        202929
OCCUPATION_TYPE     96391
CNT_FAM_MEMBERS         2
EXT_SOURCE_1       173378
EXT_SOURCE_2          660
EXT_SOURCE_3        60965
APARTMENTS_AVG     156061
dtype: int64

Shape sau khi xử lý: (307511, 122)
Số lượng null values còn lại: 8819487

Trước khi xử lý:
DAYS_EMPLOYED có giá trị 365243: 55374

Sau khi xử lý:
DAYS_EMPLOYED có giá trị 365243: 0


## 1.2. Hàm FE_application_data - Feature Engineering cho Application Data

In [15]:
def FE_application_data(data):
    """
    Feature Engineering cho Application data
    """
    print(f"Shape trước FE: {data.shape}")
    
    df = data.copy()
    
    # A. Các Tỷ Lệ Tài Chính
    df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['CREDIT_ANNUITY_PERCENT'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['FAMILY_CNT_INCOME_PERCENT'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['CHILDREN_CNT_INCOME_PERCENT'] = df['AMT_INCOME_TOTAL'] / (df['CNT_CHILDREN'] + 1)
    
    # B. Các Features Về Thời Gian
    df['CREDIT_TERM'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    df['BIRTH_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['EMPLOYED_REGISTRATION_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_REGISTRATION']
    df['BIRTH_REGISTRATION_PERCENT'] = df['DAYS_BIRTH'] / df['DAYS_REGISTRATION']
    df['ID_REGISTRATION_DIFF'] = df['DAYS_ID_PUBLISH'] - df['DAYS_REGISTRATION']
    df['ANNUITY_LENGTH_EMPLOYED_PERCENT'] = df['CREDIT_TERM'] / df['DAYS_EMPLOYED']
    
    # C. Features Về Tuổi và Thời Gian Hoàn Thành Khoản Vay
    df['AGE_LOAN_FINISH'] = df['DAYS_BIRTH'] * (-1.0/365) + (df['AMT_CREDIT']/df['AMT_ANNUITY']) * (1.0/12)
    
    # D. Features Về Xe và Điện Thoại
    df['CAR_AGE_EMP_PERCENT'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['CAR_AGE_BIRTH_PERCENT'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['PHONE_CHANGE_EMP_PERCENT'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    df['PHONE_CHANGE_BIRTH_PERCENT'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    
    # E. Features Dựa Trên Median Income Theo Nhóm
    df['MEDIAN_INCOME_CONTRACT_TYPE'] = df.groupby('NAME_CONTRACT_TYPE')['AMT_INCOME_TOTAL'].transform('median')
    df['MEDIAN_INCOME_SUITE_TYPE'] = df.groupby('NAME_TYPE_SUITE')['AMT_INCOME_TOTAL'].transform('median')
    df['MEDIAN_INCOME_HOUSING_TYPE'] = df.groupby('NAME_HOUSING_TYPE')['AMT_INCOME_TOTAL'].transform('median')
    df['MEDIAN_INCOME_ORG_TYPE'] = df.groupby('ORGANIZATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')
    df['MEDIAN_INCOME_OCCU_TYPE'] = df.groupby('OCCUPATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')
    df['MEDIAN_INCOME_EDU_TYPE'] = df.groupby('NAME_EDUCATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')
    
    # F. Tỷ Lệ Income So Với Median
    df['ORG_TYPE_INCOME_PERCENT'] = df['MEDIAN_INCOME_ORG_TYPE'] / df['AMT_INCOME_TOTAL']
    df['OCCU_TYPE_INCOME_PERCENT'] = df['MEDIAN_INCOME_OCCU_TYPE'] / df['AMT_INCOME_TOTAL']
    df['EDU_TYPE_INCOME_PERCENT'] = df['MEDIAN_INCOME_EDU_TYPE'] / df['AMT_INCOME_TOTAL']
    
    # G. Loại bỏ các cột FLAG_DOCUMENT không cần thiết
    doc_columns = [col for col in df.columns if 'FLAG_DOCUMENT' in col]
    df.drop(columns=doc_columns, inplace=True)
    
    # H. One-hot encoding cho các cột categorical
    df = pd.get_dummies(df, drop_first=False)
    
    # Thay thế inf và -inf bằng NaN, sau đó fill bằng 0
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    
    print(f"Shape sau FE: {df.shape}")
    print(f"Số features mới được tạo: {df.shape[1] - data.shape[1]}")
    
    return df

In [17]:
# Demo hàm FE_application_data
print("="*80)
print("DEMO: FE_application_data")
print("="*80)

train_fe_app = FE_application_data(train_fixed)
# test_fe_app = FE_application_data(test_fixed)

# Hiển thị một số features mới
new_features = ['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'AGE_LOAN_FINISH', 
                'MEDIAN_INCOME_ORG_TYPE', 'ORG_TYPE_INCOME_PERCENT']
print(f"\nMột số features mới được tạo:")
print(train_fe_app[new_features].head())

DEMO: FE_application_data
Shape trước FE: (307511, 122)
Shape sau FE: (307511, 250)
Số features mới được tạo: 128

Một số features mới được tạo:
   CREDIT_INCOME_PERCENT  ANNUITY_INCOME_PERCENT  AGE_LOAN_FINISH  \
0               2.007889                0.121978        27.292307   
1               4.790750                0.132217        48.951014   
2               2.000000                0.100000        53.847489   
3               2.316167                0.219900        52.946228   
4               4.222222                0.179963        56.563354   

   MEDIAN_INCOME_ORG_TYPE  ORG_TYPE_INCOME_PERCENT  
0                157500.0                 0.777778  
1                135000.0                 0.500000  
2                135000.0                 2.000000  
3                157500.0                 1.166667  
4                162000.0                 1.333333  


## 1.3. Load và xử lý Bureau data

In [18]:
print(f"Bureau shape: {bureau.shape}")
print(f"Bureau Balance shape: {bureau_balance.shape}")

Bureau shape: (1716428, 17)
Bureau Balance shape: (27299925, 3)


## 1.4. Hàm FE_bureau_data_1 - Feature Engineering Bureau Giai Đoạn 1

In [19]:
def FE_bureau_data_1(bureau_data):
    """
    Feature Engineering Bureau - Giai Đoạn 1
    """
    print(f"Shape trước FE: {bureau_data.shape}")
    
    df = bureau_data.copy()
    
    # A. Features Về Thời Gian Tín Dụng
    df['CREDIT_DURATION'] = -df['DAYS_CREDIT'] + df['DAYS_CREDIT_ENDDATE']
    df['ENDDATE_DIFF'] = df['DAYS_CREDIT_ENDDATE'] - df['DAYS_ENDDATE_FACT']
    df['UPDATE_DIFF'] = df['DAYS_CREDIT_ENDDATE'] - df['DAYS_CREDIT_UPDATE']
    
    # B. Features Về Nợ và Tỷ Lệ
    df['DEBT_PERCENTAGE'] = df['AMT_CREDIT_SUM'] / (df['AMT_CREDIT_SUM_DEBT'] + 1)
    df['DEBT_CREDIT_DIFF'] = df['AMT_CREDIT_SUM'] - df['AMT_CREDIT_SUM_DEBT']
    df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT_SUM'] / (df['AMT_ANNUITY'] + 1)
    df['DEBT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT_SUM_DEBT'] / (df['AMT_ANNUITY'] + 1)
    df['CREDIT_OVERDUE_DIFF'] = df['AMT_CREDIT_SUM'] - df['AMT_CREDIT_SUM_OVERDUE']
    
    # C. Features Về Lịch Sử Vay
    df['CUSTOMER_LOAN_COUNT'] = df.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')
    df['CUSTOMER_CREDIT_TYPES'] = df.groupby('SK_ID_CURR')['CREDIT_TYPE'].transform('nunique')
    df['AVG_LOAN_TYPE'] = df['CUSTOMER_LOAN_COUNT'] / (df['CUSTOMER_CREDIT_TYPES'] + 1)
    
    # D. Credit Type Code
    credit_type_mapping = {'Closed': 0, 'Active': 1}
    df['CREDIT_TYPE_CODE'] = df['CREDIT_ACTIVE'].map(credit_type_mapping).fillna(2)
    
    # E. Tổng Hợp Credit và Debt
    df['TOTAL_CREDIT_SUM'] = df.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('sum')
    df['TOTAL_DEBT_SUM'] = df.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform('sum')
    df['CREDIT_DEBT_RATIO'] = df['TOTAL_CREDIT_SUM'] / (df['TOTAL_DEBT_SUM'] + 1)
    
    # One-hot encoding
    df = pd.get_dummies(df, drop_first=False)
    
    # Xử lý inf và NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    
    print(f"Shape sau FE: {df.shape}")
    print(f"Số features mới: {df.shape[1] - bureau_data.shape[1]}")
    
    return df

In [20]:
# Demo hàm FE_bureau_data_1
print("="*80)
print("DEMO: FE_bureau_data_1")
print("="*80)

bureau_fe_1 = FE_bureau_data_1(bureau)

# Hiển thị một số features mới
new_features = ['CREDIT_DURATION', 'DEBT_PERCENTAGE', 'CUSTOMER_LOAN_COUNT', 
                'AVG_LOAN_TYPE', 'CREDIT_DEBT_RATIO']
print(f"\nMột số features mới từ Bureau:")
print(bureau_fe_1[['SK_ID_CURR'] + new_features].head())

DEMO: FE_bureau_data_1
Shape trước FE: (1716428, 17)
Shape sau FE: (1716428, 52)
Số features mới: 35

Một số features mới từ Bureau:
   SK_ID_CURR  CREDIT_DURATION  DEBT_PERCENTAGE  CUSTOMER_LOAN_COUNT  \
0      215354            344.0     91323.000000                   11   
1      215354           1283.0         1.313155                   11   
2      215354            731.0         0.000000                   11   
3      215354              0.0         0.000000                   11   
4      215354           1826.0         0.000000                   11   

   AVG_LOAN_TYPE  CREDIT_DEBT_RATIO  
0           2.75          21.000694  
1           2.75          21.000694  
2           2.75          21.000694  
3           2.75          21.000694  
4           2.75          21.000694  


## 1.5. Hàm FE_bureau_data_2 - Aggregation và Merge Bureau Balance

In [21]:
def FE_bureau_data_2(bureau_data, bureau_balance_data):
    """
    Feature Engineering Bureau - Giai Đoạn 2: Aggregation
    """
    print(f"Bureau shape trước aggregation: {bureau_data.shape}")
    
    # A. Aggregation Bureau Balance theo SK_ID_BUREAU
    bb_agg = bureau_balance_data.groupby('SK_ID_BUREAU').agg({
        'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']
    })
    bb_agg.columns = ['BB_' + '_'.join(col).upper() for col in bb_agg.columns]
    bb_agg.reset_index(inplace=True)
    
    # B. Join Bureau với Bureau Balance
    bureau_merged = bureau_data.merge(bb_agg, on='SK_ID_BUREAU', how='left')
    
    # C. Numerical aggregation theo SK_ID_CURR
    num_agg = {
        'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean', 'sum'],
        'DAYS_CREDIT': ['mean', 'var'],
        'DAYS_CREDIT_UPDATE': ['mean', 'min'],
        'CREDIT_DAY_OVERDUE': ['mean', 'min'],
        'DAYS_CREDIT_ENDDATE': ['mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'BB_MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
        'AMT_ANNUITY': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM': ['mean', 'sum', 'max']
    }
    
    bureau_agg = bureau_merged.groupby('SK_ID_CURR').agg(num_agg)
    bureau_agg.columns = ['BUREAU_' + '_'.join(col).upper() for col in bureau_agg.columns]
    bureau_agg.reset_index(inplace=True)
    
    # D. Tách Active và Closed Credits
    # Active Credits
    active_cols = [col for col in bureau_merged.columns if 'CREDIT_ACTIVE_Active' in col]
    if len(active_cols) > 0:
        active = bureau_merged[bureau_merged[active_cols[0]] == 1]
        active_agg = active.groupby('SK_ID_CURR').agg(num_agg)
        active_agg.columns = ['A_BUREAU_' + '_'.join(col).upper() for col in active_agg.columns]
        active_agg.reset_index(inplace=True)
        bureau_agg = bureau_agg.merge(active_agg, on='SK_ID_CURR', how='left')
    
    # Closed Credits
    closed_cols = [col for col in bureau_merged.columns if 'CREDIT_ACTIVE_Closed' in col]
    if len(closed_cols) > 0:
        closed = bureau_merged[bureau_merged[closed_cols[0]] == 1]
        closed_agg = closed.groupby('SK_ID_CURR').agg(num_agg)
        closed_agg.columns = ['C_BUREAU_' + '_'.join(col).upper() for col in closed_agg.columns]
        closed_agg.reset_index(inplace=True)
        bureau_agg = bureau_agg.merge(closed_agg, on='SK_ID_CURR', how='left')
    
    bureau_agg.fillna(0, inplace=True)
    
    print(f"Bureau aggregated shape: {bureau_agg.shape}")
    
    return bureau_agg

In [22]:
# Demo hàm FE_bureau_data_2
print("="*80)
print("DEMO: FE_bureau_data_2")
print("="*80)

bureau_agg = FE_bureau_data_2(bureau_fe_1, bureau_balance)

print(f"\nCác cột aggregated từ Bureau:")
print(bureau_agg.columns.tolist()[:20])
print(f"\nData sample:")
print(bureau_agg.head())

DEMO: FE_bureau_data_2
Bureau shape trước aggregation: (1716428, 52)
Bureau aggregated shape: (305811, 73)

Các cột aggregated từ Bureau:
['SK_ID_CURR', 'BUREAU_AMT_CREDIT_SUM_DEBT_MEAN', 'BUREAU_AMT_CREDIT_SUM_DEBT_SUM', 'BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN', 'BUREAU_AMT_CREDIT_SUM_OVERDUE_SUM', 'BUREAU_DAYS_CREDIT_MEAN', 'BUREAU_DAYS_CREDIT_VAR', 'BUREAU_DAYS_CREDIT_UPDATE_MEAN', 'BUREAU_DAYS_CREDIT_UPDATE_MIN', 'BUREAU_CREDIT_DAY_OVERDUE_MEAN', 'BUREAU_CREDIT_DAY_OVERDUE_MIN', 'BUREAU_DAYS_CREDIT_ENDDATE_MEAN', 'BUREAU_CNT_CREDIT_PROLONG_SUM', 'BUREAU_BB_MONTHS_BALANCE_SIZE_MEAN', 'BUREAU_BB_MONTHS_BALANCE_SIZE_SUM', 'BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN', 'BUREAU_AMT_CREDIT_SUM_LIMIT_SUM', 'BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN', 'BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX', 'BUREAU_AMT_ANNUITY_MAX']

Data sample:
   SK_ID_CURR  BUREAU_AMT_CREDIT_SUM_DEBT_MEAN  \
0      100001                     85240.928571   
1      100002                     30722.625000   
2      100003                        

## 1.6. Merge Bureau features vào Application data

In [23]:
# Merge Bureau features vào train và test
train_with_bureau = train_fe_app.merge(bureau_agg, on='SK_ID_CURR', how='left')
# test_with_bureau = test_fe_app.merge(bureau_agg, on='SK_ID_CURR', how='left')

print(f"Train shape sau khi merge Bureau: {train_with_bureau.shape}")
# print(f"Test shape sau khi merge Bureau: {test_with_bureau.shape}")

Train shape sau khi merge Bureau: (307511, 322)


## 1.7. Feature Engineering cho Previous Application

Tương tự, chúng ta sẽ làm feature engineering cho các bảng khác. Do notebook khá dài, tôi sẽ triển khai các hàm còn lại trong phần pipeline.

# PHẦN 2: SKLEARN PIPELINE CHO FEATURE ENGINEERING

Trong phần này, chúng ta sẽ viết toàn bộ feature engineering dưới dạng sklearn pipeline để dễ dàng tái sử dụng.

## 2.1. Import thêm các thư viện cần thiết cho Pipeline

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer

## 2.2. Custom Transformers cho từng bước

In [25]:
class NullOutlierFixer(BaseEstimator, TransformerMixin):
    """
    Transformer để xử lý null và outliers trong Application data
    """
    def __init__(self):
        self.mode_values = {}
        self.median_values = {}
    
    def fit(self, X, y=None):
        # Lưu mode và median từ training data
        self.mode_values['CNT_FAM_MEMBERS'] = X['CNT_FAM_MEMBERS'].mode()[0]
        self.median_values['DAYS_EMPLOYED'] = X[X['DAYS_EMPLOYED'] != 365243]['DAYS_EMPLOYED'].median()
        return self
    
    def transform(self, X):
        df = X.copy()
        
        # Xử lý categorical
        df['NAME_FAMILY_STATUS'].fillna('Data_Not_Available', inplace=True)
        df['NAME_HOUSING_TYPE'].fillna('Data_Not_Available', inplace=True)
        df['FLAG_MOBIL'].fillna('Data_Not_Available', inplace=True)
        df['FLAG_EMP_PHONE'].fillna('Data_Not_Available', inplace=True)
        df['FLAG_CONT_MOBILE'].fillna('Data_Not_Available', inplace=True)
        df['FLAG_EMAIL'].fillna('Data_Not_Available', inplace=True)
        df['OCCUPATION_TYPE'].fillna('Data_Not_Available', inplace=True)
        df['CNT_FAM_MEMBERS'].fillna(self.mode_values['CNT_FAM_MEMBERS'], inplace=True)
        df.loc[df['CODE_GENDER'] == 'XNA', 'CODE_GENDER'] = 'M'
        df['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace=True)
        df.loc[df['NAME_FAMILY_STATUS'] == 'Unknown', 'NAME_FAMILY_STATUS'] = 'Married'
        
        # Xử lý numerical
        df.loc[df['DAYS_EMPLOYED'] == 365243, 'DAYS_EMPLOYED'] = np.nan
        df['DAYS_EMPLOYED'].fillna(self.median_values['DAYS_EMPLOYED'], inplace=True)
        df['AMT_ANNUITY'].fillna(0, inplace=True)
        df['AMT_GOODS_PRICE'].fillna(0, inplace=True)
        df['EXT_SOURCE_1'].fillna(0, inplace=True)
        df['EXT_SOURCE_2'].fillna(0, inplace=True)
        df['EXT_SOURCE_3'].fillna(0, inplace=True)
        
        return df


class ApplicationFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Transformer để tạo features cho Application data
    """
    def __init__(self):
        self.categorical_cols = None
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        
        # A. Các Tỷ Lệ Tài Chính
        df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / (df['AMT_INCOME_TOTAL'] + 1)
        df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / (df['AMT_INCOME_TOTAL'] + 1)
        df['CREDIT_ANNUITY_PERCENT'] = df['AMT_CREDIT'] / (df['AMT_ANNUITY'] + 1)
        df['FAMILY_CNT_INCOME_PERCENT'] = df['AMT_INCOME_TOTAL'] / (df['CNT_FAM_MEMBERS'] + 1)
        df['CHILDREN_CNT_INCOME_PERCENT'] = df['AMT_INCOME_TOTAL'] / (df['CNT_CHILDREN'] + 1)
        
        # B. Các Features Về Thời Gian
        df['CREDIT_TERM'] = df['AMT_ANNUITY'] / (df['AMT_CREDIT'] + 1)
        df['BIRTH_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / (df['DAYS_BIRTH'] + 1)
        df['EMPLOYED_REGISTRATION_PERCENT'] = df['DAYS_EMPLOYED'] / (df['DAYS_REGISTRATION'] + 1)
        df['BIRTH_REGISTRATION_PERCENT'] = df['DAYS_BIRTH'] / (df['DAYS_REGISTRATION'] + 1)
        df['ID_REGISTRATION_DIFF'] = df['DAYS_ID_PUBLISH'] - df['DAYS_REGISTRATION']
        df['ANNUITY_LENGTH_EMPLOYED_PERCENT'] = df['CREDIT_TERM'] / (df['DAYS_EMPLOYED'] + 1)
        
        # C. Features Về Tuổi và Thời Gian Hoàn Thành Khoản Vay
        df['AGE_LOAN_FINISH'] = df['DAYS_BIRTH'] * (-1.0/365) + (df['AMT_CREDIT']/(df['AMT_ANNUITY'] + 1)) * (1.0/12)
        
        # D. Features Về Xe và Điện Thoại
        df['CAR_AGE_EMP_PERCENT'] = df['OWN_CAR_AGE'] / (df['DAYS_EMPLOYED'] + 1)
        df['CAR_AGE_BIRTH_PERCENT'] = df['OWN_CAR_AGE'] / (df['DAYS_BIRTH'] + 1)
        df['PHONE_CHANGE_EMP_PERCENT'] = df['DAYS_LAST_PHONE_CHANGE'] / (df['DAYS_EMPLOYED'] + 1)
        df['PHONE_CHANGE_BIRTH_PERCENT'] = df['DAYS_LAST_PHONE_CHANGE'] / (df['DAYS_BIRTH'] + 1)
        
        # E. Features Dựa Trên Median Income Theo Nhóm
        df['MEDIAN_INCOME_CONTRACT_TYPE'] = df.groupby('NAME_CONTRACT_TYPE')['AMT_INCOME_TOTAL'].transform('median')
        df['MEDIAN_INCOME_SUITE_TYPE'] = df.groupby('NAME_TYPE_SUITE')['AMT_INCOME_TOTAL'].transform('median')
        df['MEDIAN_INCOME_HOUSING_TYPE'] = df.groupby('NAME_HOUSING_TYPE')['AMT_INCOME_TOTAL'].transform('median')
        df['MEDIAN_INCOME_ORG_TYPE'] = df.groupby('ORGANIZATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')
        df['MEDIAN_INCOME_OCCU_TYPE'] = df.groupby('OCCUPATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')
        df['MEDIAN_INCOME_EDU_TYPE'] = df.groupby('NAME_EDUCATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')
        
        # F. Tỷ Lệ Income So Với Median
        df['ORG_TYPE_INCOME_PERCENT'] = df['MEDIAN_INCOME_ORG_TYPE'] / (df['AMT_INCOME_TOTAL'] + 1)
        df['OCCU_TYPE_INCOME_PERCENT'] = df['MEDIAN_INCOME_OCCU_TYPE'] / (df['AMT_INCOME_TOTAL'] + 1)
        df['EDU_TYPE_INCOME_PERCENT'] = df['MEDIAN_INCOME_EDU_TYPE'] / (df['AMT_INCOME_TOTAL'] + 1)
        
        # G. Loại bỏ các cột FLAG_DOCUMENT
        doc_columns = [col for col in df.columns if 'FLAG_DOCUMENT' in col]
        df.drop(columns=doc_columns, inplace=True)
        
        # H. One-hot encoding
        df = pd.get_dummies(df, drop_first=False)
        
        # Thay thế inf và NaN
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.fillna(0, inplace=True)
        
        return df


class BureauFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Transformer để xử lý Bureau và Bureau Balance data
    """
    def __init__(self, bureau_path, bureau_balance_path):
        self.bureau_path = bureau_path
        self.bureau_balance_path = bureau_balance_path
        self.bureau_features = None
        
    def fit(self, X, y=None):
        # Load bureau data
        bureau = pd.read_csv(self.bureau_path)
        bureau_balance = pd.read_csv(self.bureau_balance_path)
        
        # Feature Engineering Giai Đoạn 1
        bureau = self._fe_bureau_stage1(bureau)
        
        # Feature Engineering Giai Đoạn 2: Aggregation
        self.bureau_features = self._fe_bureau_stage2(bureau, bureau_balance)
        
        return self
    
    def transform(self, X):
        df = X.copy()
        
        # Merge bureau features
        df = df.merge(self.bureau_features, on='SK_ID_CURR', how='left')
        df.fillna(0, inplace=True)
        
        return df
    
    def _fe_bureau_stage1(self, bureau):
        """Feature Engineering Bureau - Giai Đoạn 1"""
        df = bureau.copy()
        
        # Features Về Thời Gian Tín Dụng
        df['CREDIT_DURATION'] = -df['DAYS_CREDIT'] + df['DAYS_CREDIT_ENDDATE']
        df['ENDDATE_DIFF'] = df['DAYS_CREDIT_ENDDATE'] - df['DAYS_ENDDATE_FACT']
        df['UPDATE_DIFF'] = df['DAYS_CREDIT_ENDDATE'] - df['DAYS_CREDIT_UPDATE']
        
        # Features Về Nợ và Tỷ Lệ
        df['DEBT_PERCENTAGE'] = df['AMT_CREDIT_SUM'] / (df['AMT_CREDIT_SUM_DEBT'] + 1)
        df['DEBT_CREDIT_DIFF'] = df['AMT_CREDIT_SUM'] - df['AMT_CREDIT_SUM_DEBT']
        df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT_SUM'] / (df['AMT_ANNUITY'] + 1)
        df['DEBT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT_SUM_DEBT'] / (df['AMT_ANNUITY'] + 1)
        df['CREDIT_OVERDUE_DIFF'] = df['AMT_CREDIT_SUM'] - df['AMT_CREDIT_SUM_OVERDUE']
        
        # Features Về Lịch Sử Vay
        df['CUSTOMER_LOAN_COUNT'] = df.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')
        df['CUSTOMER_CREDIT_TYPES'] = df.groupby('SK_ID_CURR')['CREDIT_TYPE'].transform('nunique')
        df['AVG_LOAN_TYPE'] = df['CUSTOMER_LOAN_COUNT'] / (df['CUSTOMER_CREDIT_TYPES'] + 1)
        
        # Credit Type Code
        credit_type_mapping = {'Closed': 0, 'Active': 1}
        df['CREDIT_TYPE_CODE'] = df['CREDIT_ACTIVE'].map(credit_type_mapping).fillna(2)
        
        # Tổng Hợp Credit và Debt
        df['TOTAL_CREDIT_SUM'] = df.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('sum')
        df['TOTAL_DEBT_SUM'] = df.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform('sum')
        df['CREDIT_DEBT_RATIO'] = df['TOTAL_CREDIT_SUM'] / (df['TOTAL_DEBT_SUM'] + 1)
        
        # One-hot encoding
        df = pd.get_dummies(df, drop_first=False)
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.fillna(0, inplace=True)
        
        return df
    
    def _fe_bureau_stage2(self, bureau, bureau_balance):
        """Feature Engineering Bureau - Giai Đoạn 2: Aggregation"""
        
        # Aggregation Bureau Balance
        bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg({
            'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']
        })
        bb_agg.columns = ['BB_' + '_'.join(col).upper() for col in bb_agg.columns]
        bb_agg.reset_index(inplace=True)
        
        # Merge Bureau với Bureau Balance
        bureau = bureau.merge(bb_agg, on='SK_ID_BUREAU', how='left')
        
        # Numerical aggregation
        num_agg = {
            'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
            'AMT_CREDIT_SUM_OVERDUE': ['mean', 'sum'],
            'DAYS_CREDIT': ['mean', 'var'],
            'DAYS_CREDIT_UPDATE': ['mean', 'min'],
            'CREDIT_DAY_OVERDUE': ['mean', 'min'],
            'DAYS_CREDIT_ENDDATE': ['mean'],
            'CNT_CREDIT_PROLONG': ['sum'],
            'BB_MONTHS_BALANCE_SIZE': ['mean', 'sum'],
            'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
            'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
            'AMT_ANNUITY': ['max', 'mean', 'sum'],
            'AMT_CREDIT_SUM': ['mean', 'sum', 'max']
        }
        
        bureau_agg = bureau.groupby('SK_ID_CURR').agg(num_agg)
        bureau_agg.columns = ['BUREAU_' + '_'.join(col).upper() for col in bureau_agg.columns]
        bureau_agg.reset_index(inplace=True)
        
        # Active và Closed credits
        active_cols = [col for col in bureau.columns if 'CREDIT_ACTIVE_Active' in col]
        if len(active_cols) > 0:
            active = bureau[bureau[active_cols[0]] == 1]
            active_agg = active.groupby('SK_ID_CURR').agg(num_agg)
            active_agg.columns = ['A_BUREAU_' + '_'.join(col).upper() for col in active_agg.columns]
            active_agg.reset_index(inplace=True)
            bureau_agg = bureau_agg.merge(active_agg, on='SK_ID_CURR', how='left')
        
        closed_cols = [col for col in bureau.columns if 'CREDIT_ACTIVE_Closed' in col]
        if len(closed_cols) > 0:
            closed = bureau[bureau[closed_cols[0]] == 1]
            closed_agg = closed.groupby('SK_ID_CURR').agg(num_agg)
            closed_agg.columns = ['C_BUREAU_' + '_'.join(col).upper() for col in closed_agg.columns]
            closed_agg.reset_index(inplace=True)
            bureau_agg = bureau_agg.merge(closed_agg, on='SK_ID_CURR', how='left')
        
        bureau_agg.fillna(0, inplace=True)
        
        return bureau_agg

In [26]:
class PreviousApplicationFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Transformer để xử lý Previous Application data
    """
    def __init__(self, previous_app_path):
        self.previous_app_path = previous_app_path
        self.prev_features = None
        
    def fit(self, X, y=None):
        # Load previous application
        prev_app = pd.read_csv(self.previous_app_path)
        
        # Xử lý outliers
        prev_app.loc[prev_app['DAYS_FIRST_DRAWING'] == 365243, 'DAYS_FIRST_DRAWING'] = np.nan
        prev_app.loc[prev_app['DAYS_FIRST_DUE'] == 365243, 'DAYS_FIRST_DUE'] = np.nan
        prev_app.loc[prev_app['DAYS_LAST_DUE_1ST_VERSION'] == 365243, 'DAYS_LAST_DUE_1ST_VERSION'] = np.nan
        prev_app.loc[prev_app['DAYS_LAST_DUE'] == 365243, 'DAYS_LAST_DUE'] = np.nan
        prev_app.loc[prev_app['DAYS_TERMINATION'] == 365243, 'DAYS_TERMINATION'] = np.nan
        
        # Feature Engineering
        prev_app['APPLICATION_CREDIT_DIFF'] = prev_app['AMT_APPLICATION'] - prev_app['AMT_CREDIT']
        prev_app['APPLICATION_CREDIT_RATIO'] = prev_app['AMT_APPLICATION'] / (prev_app['AMT_CREDIT'] + 1)
        prev_app['CREDIT_TO_ANNUITY_RATIO'] = prev_app['AMT_CREDIT'] / (prev_app['AMT_ANNUITY'] + 1)
        prev_app['DOWN_PAYMENT_TO_CREDIT'] = prev_app['AMT_DOWN_PAYMENT'] / (prev_app['AMT_CREDIT'] + 1)
        
        total_payment = prev_app['AMT_ANNUITY'] * prev_app['CNT_PAYMENT']
        prev_app['SIMPLE_INTERESTS'] = (total_payment / (prev_app['AMT_CREDIT'] + 1) - 1) / (prev_app['CNT_PAYMENT'] + 1)
        
        prev_app['DAYS_LAST_DUE_DIFF'] = prev_app['DAYS_LAST_DUE_1ST_VERSION'] - prev_app['DAYS_LAST_DUE']
        
        # One-hot encoding
        prev_app = pd.get_dummies(prev_app, drop_first=False)
        prev_app.replace([np.inf, -np.inf], np.nan, inplace=True)
        prev_app.fillna(0, inplace=True)
        
        # Aggregation
        num_agg = {
            'AMT_ANNUITY': ['max', 'mean'],
            'AMT_APPLICATION': ['max', 'mean'],
            'AMT_CREDIT': ['max', 'mean'],
            'AMT_DOWN_PAYMENT': ['max', 'mean'],
            'AMT_GOODS_PRICE': ['mean', 'sum'],
            'HOUR_APPR_PROCESS_START': ['max', 'mean'],
            'RATE_DOWN_PAYMENT': ['max', 'mean'],
            'RATE_INTEREST_PRIMARY': ['max', 'mean'],
            'RATE_INTEREST_PRIVILEGED': ['max', 'mean'],
            'DAYS_DECISION': ['max', 'mean'],
            'CNT_PAYMENT': ['mean', 'sum'],
            'APPLICATION_CREDIT_RATIO': ['max', 'mean'],
            'DOWN_PAYMENT_TO_CREDIT': ['max', 'mean'],
            'DAYS_LAST_DUE_DIFF': ['max', 'mean']
        }
        
        prev_agg = prev_app.groupby('SK_ID_CURR').agg(num_agg)
        prev_agg.columns = ['PREV_' + '_'.join(col).upper() for col in prev_agg.columns]
        prev_agg.reset_index(inplace=True)
        
        # Approved/Refused
        approved_col = [col for col in prev_app.columns if 'NAME_CONTRACT_STATUS_Approved' in col]
        if len(approved_col) > 0:
            approved = prev_app[prev_app[approved_col[0]] == 1]
            approved_agg = approved.groupby('SK_ID_CURR').agg(num_agg)
            approved_agg.columns = ['CS_APP_' + '_'.join(col).upper() for col in approved_agg.columns]
            approved_agg.reset_index(inplace=True)
            prev_agg = prev_agg.merge(approved_agg, on='SK_ID_CURR', how='left')
        
        refused_col = [col for col in prev_app.columns if 'NAME_CONTRACT_STATUS_Refused' in col]
        if len(refused_col) > 0:
            refused = prev_app[prev_app[refused_col[0]] == 1]
            refused_agg = refused.groupby('SK_ID_CURR').agg(num_agg)
            refused_agg.columns = ['CS_REF_' + '_'.join(col).upper() for col in refused_agg.columns]
            refused_agg.reset_index(inplace=True)
            prev_agg = prev_agg.merge(refused_agg, on='SK_ID_CURR', how='left')
        
        prev_agg.fillna(0, inplace=True)
        self.prev_features = prev_agg
        
        return self
    
    def transform(self, X):
        df = X.copy()
        df = df.merge(self.prev_features, on='SK_ID_CURR', how='left')
        df.fillna(0, inplace=True)
        return df


class POSCashFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Transformer để xử lý POS Cash Balance data
    """
    def __init__(self, pos_cash_path):
        self.pos_cash_path = pos_cash_path
        self.pos_features = None
        
    def fit(self, X, y=None):
        pos = pd.read_csv(self.pos_cash_path)
        
        # Feature Engineering
        pos['LATE_PAYMENT'] = (pos['SK_DPD'] > 0).astype(int)
        
        # One-hot encoding
        pos = pd.get_dummies(pos, drop_first=False)
        pos.replace([np.inf, -np.inf], np.nan, inplace=True)
        pos.fillna(0, inplace=True)
        
        # Aggregation
        num_agg = {
            'SK_DPD_DEF': ['max', 'mean', 'min'],
            'SK_DPD': ['max', 'mean', 'min'],
            'MONTHS_BALANCE': ['max', 'mean', 'size'],
            'CNT_INSTALMENT': ['max', 'size'],
            'CNT_INSTALMENT_FUTURE': ['max', 'size', 'sum'],
            'LATE_PAYMENT': ['mean', 'sum']
        }
        
        pos_agg = pos.groupby('SK_ID_CURR').agg(num_agg)
        pos_agg.columns = ['POS_' + '_'.join(col).upper() for col in pos_agg.columns]
        pos_agg.reset_index(inplace=True)
        pos_agg.fillna(0, inplace=True)
        
        self.pos_features = pos_agg
        
        return self
    
    def transform(self, X):
        df = X.copy()
        df = df.merge(self.pos_features, on='SK_ID_CURR', how='left')
        df.fillna(0, inplace=True)
        return df


class InstallmentsPaymentsFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Transformer để xử lý Installments Payments data
    """
    def __init__(self, installments_path):
        self.installments_path = installments_path
        self.ins_features = None
        
    def fit(self, X, y=None):
        ins = pd.read_csv(self.installments_path)
        
        # Group payments
        ins_grouped = ins.groupby(['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'])['AMT_PAYMENT'].sum().reset_index()
        ins_grouped.rename(columns={'AMT_PAYMENT': 'AMT_PAYMENT_GROUPED'}, inplace=True)
        
        ins = ins.merge(ins_grouped, on=['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], how='left')
        
        # Features
        ins['PAYMENT_DIFFERENCE'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT_GROUPED']
        ins['PAYMENT_RATIO'] = ins['AMT_INSTALMENT'] / (ins['AMT_PAYMENT_GROUPED'] + 1)
        ins['PAID_OVER_AMOUNT'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT']
        ins['PAID_OVER'] = (ins['PAID_OVER_AMOUNT'] > 0).astype(int)
        
        # DPD features
        ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
        ins.loc[ins['DPD'] <= 0, 'DPD'] = 0
        
        ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
        ins.loc[ins['DBD'] <= 0, 'DBD'] = 0
        ins['LATE_PAYMENT'] = (ins['DBD'] > 0).astype(int)
        
        ins['INSTALMENT_PAYMENT_RATIO'] = ins['AMT_PAYMENT'] / (ins['AMT_INSTALMENT'] + 1)
        ins['LATE_PAYMENT_RATIO'] = ins['INSTALMENT_PAYMENT_RATIO'] * ins['LATE_PAYMENT']
        ins['SIGNIFICANT_LATE_PAYMENT'] = (ins['LATE_PAYMENT_RATIO'] > 0.05).astype(int)
        
        # DPD Buckets
        ins['DPD_7'] = (ins['DPD'] >= 7).astype(int)
        ins['DPD_15'] = (ins['DPD'] >= 15).astype(int)
        ins['DPD_30'] = (ins['DPD'] >= 30).astype(int)
        ins['DPD_60'] = (ins['DPD'] >= 60).astype(int)
        ins['DPD_90'] = (ins['DPD'] >= 90).astype(int)
        ins['DPD_180'] = (ins['DPD'] >= 180).astype(int)
        ins['DPD_WOF'] = (ins['DPD'] >= 720).astype(int)
        
        ins.replace([np.inf, -np.inf], np.nan, inplace=True)
        ins.fillna(0, inplace=True)
        
        # Aggregation
        num_agg = {
            'LATE_PAYMENT': ['max', 'mean', 'min'],
            'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
            'NUM_INSTALMENT_VERSION': ['nunique'],
            'NUM_INSTALMENT_NUMBER': ['max'],
            'AMT_INSTALMENT': ['max', 'mean', 'sum'],
            'PAYMENT_DIFFERENCE': ['max', 'mean', 'min', 'sum'],
            'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum'],
            'PAID_OVER_AMOUNT': ['max', 'mean', 'min'],
            'DPD': ['max', 'mean', 'sum'],
            'DPD_7': ['mean', 'sum'],
            'DPD_15': ['mean', 'sum'],
            'DPD_30': ['mean', 'sum'],
            'DPD_60': ['mean', 'sum'],
            'DPD_90': ['mean', 'sum'],
            'DPD_180': ['mean', 'sum'],
            'DPD_WOF': ['mean', 'sum']
        }
        
        ins_agg = ins.groupby('SK_ID_CURR').agg(num_agg)
        ins_agg.columns = ['INS_' + '_'.join(col).upper() for col in ins_agg.columns]
        ins_agg.reset_index(inplace=True)
        ins_agg.fillna(0, inplace=True)
        
        self.ins_features = ins_agg
        
        return self
    
    def transform(self, X):
        df = X.copy()
        df = df.merge(self.ins_features, on='SK_ID_CURR', how='left')
        df.fillna(0, inplace=True)
        return df


class CreditCardBalanceFeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Transformer để xử lý Credit Card Balance data
    """
    def __init__(self, credit_card_path):
        self.credit_card_path = credit_card_path
        self.cc_features = None
        
    def fit(self, X, y=None):
        cc = pd.read_csv(self.credit_card_path)
        
        # Features
        cc['LIMIT_USE'] = cc['AMT_BALANCE'] / (cc['AMT_CREDIT_LIMIT_ACTUAL'] + 1)
        cc['PAYMENT_DIV_MIN'] = cc['AMT_PAYMENT_CURRENT'] / (cc['AMT_INST_MIN_REGULARITY'] + 1)
        cc['LATE_PAYMENT'] = (cc['SK_DPD'] > 0).astype(int)
        cc['DRAWING_LIMIT_RATIO'] = cc['AMT_DRAWINGS_ATM_CURRENT'] / (cc['AMT_CREDIT_LIMIT_ACTUAL'] + 1)
        
        cc.replace([np.inf, -np.inf], np.nan, inplace=True)
        cc.fillna(0, inplace=True)
        
        # Aggregation
        num_cols = cc.select_dtypes(include=[np.number]).columns.tolist()
        num_cols.remove('SK_ID_PREV')
        num_cols.remove('SK_ID_CURR')
        
        num_agg = {col: ['max', 'mean', 'sum', 'var'] for col in num_cols}
        
        cc_agg = cc.groupby('SK_ID_CURR').agg(num_agg)
        cc_agg.columns = ['CR_' + '_'.join(col).upper() for col in cc_agg.columns]
        cc_agg.reset_index(inplace=True)
        cc_agg.fillna(0, inplace=True)
        
        self.cc_features = cc_agg
        
        return self
    
    def transform(self, X):
        df = X.copy()
        df = df.merge(self.cc_features, on='SK_ID_CURR', how='left')
        df.fillna(0, inplace=True)
        return df

## 2.3. Tạo Complete Feature Engineering Pipeline

In [27]:
def create_feature_engineering_pipeline(data_path='home-credit-default-risk/'):
    """
    Tạo complete pipeline cho feature engineering
    
    Parameters:
    -----------
    data_path : str
        Đường dẫn đến thư mục chứa data files
    
    Returns:
    --------
    pipeline : Pipeline
        Sklearn pipeline hoàn chỉnh
    """
    
    pipeline = Pipeline([
        ('null_outlier_fixer', NullOutlierFixer()),
        ('app_feature_engineer', ApplicationFeatureEngineer()),
        ('bureau_feature_engineer', BureauFeatureEngineer(
            bureau_path=data_path + 'bureau.csv',
            bureau_balance_path=data_path + 'bureau_balance.csv'
        )),
        ('prev_app_feature_engineer', PreviousApplicationFeatureEngineer(
            previous_app_path=data_path + 'previous_application.csv'
        )),
        ('pos_cash_feature_engineer', POSCashFeatureEngineer(
            pos_cash_path=data_path + 'POS_CASH_balance.csv'
        )),
        ('installments_feature_engineer', InstallmentsPaymentsFeatureEngineer(
            installments_path=data_path + 'installments_payments.csv'
        )),
        ('credit_card_feature_engineer', CreditCardBalanceFeatureEngineer(
            credit_card_path=data_path + 'credit_card_balance.csv'
        ))
    ])
    
    return pipeline

## 2.4. Demo sử dụng Pipeline

In [33]:
# Tạo pipeline
print("="*80)
print("DEMO: SKLEARN FEATURE ENGINEERING PIPELINE")
print("="*80)

# Sử dụng đường dẫn tuyệt đối hoặc đường dẫn đúng từ workspace root
data_path = "C:\\Users\\SonNT\\Downloads\\Internship2\\home-credit-default-risk\\"
print(f"Data path: {data_path}")

fe_pipeline = create_feature_engineering_pipeline(data_path=data_path)
print("✓ Pipeline đã được khởi tạo")

# Fit và transform train data
print("\n1. Fitting và transforming train data...")
print(f"   Input shape: {application_train.shape}")

try:
    # Sử dụng fit_transform() để fit và transform trong một bước
    train_transformed = fe_pipeline.fit_transform(application_train)
    print("✓ Pipeline đã được fit và transform thành công!")
    print(f"✓ Train shape sau transform: {train_transformed.shape}")
except Exception as e:
    print(f"✗ Lỗi khi fit/transform pipeline: {e}")
    import traceback
    traceback.print_exc()
    raise

# Transform test data (nếu có)
# print("\n2. Transforming test data...")
# test_transformed = fe_pipeline.transform(test_data)
# print(f"✓ Test shape sau transform: {test_transformed.shape}")

print("\n" + "="*80)
print("✓ HOÀN THÀNH FEATURE ENGINEERING!")
print("="*80)
print(f"✓ Tổng số features: {train_transformed.shape[1]}")
print(f"✓ Số features mới được tạo: {train_transformed.shape[1] - application_train.shape[1]}")
print(f"✓ Tỷ lệ tăng: {((train_transformed.shape[1] / application_train.shape[1]) - 1) * 100:.1f}%")

DEMO: SKLEARN FEATURE ENGINEERING PIPELINE
Data path: C:\Users\SonNT\Downloads\Internship2\home-credit-default-risk\
✓ Pipeline đã được khởi tạo

1. Fitting và transforming train data...
   Input shape: (307511, 122)
✓ Pipeline đã được fit và transform thành công!
✓ Train shape sau transform: (307511, 557)

✓ HOÀN THÀNH FEATURE ENGINEERING!
✓ Tổng số features: 557
✓ Số features mới được tạo: 435
✓ Tỷ lệ tăng: 356.6%


## 2.5. Lưu Pipeline để tái sử dụng

In [34]:
# Lưu pipeline
import joblib

# Lưu fitted pipeline
joblib.dump(fe_pipeline, 'feature_engineering_pipeline.pkl')
print("✓ Pipeline đã được lưu vào 'feature_engineering_pipeline.pkl'")

# Để load lại pipeline:
# fe_pipeline_loaded = joblib.load('feature_engineering_pipeline.pkl')
# new_data_transformed = fe_pipeline_loaded.transform(new_data)

✓ Pipeline đã được lưu vào 'feature_engineering_pipeline.pkl'


## 2.6. So sánh trước và sau Feature Engineering

In [35]:
# So sánh shape
print("="*80)
print("SO SÁNH TRƯỚC VÀ SAU FEATURE ENGINEERING")
print("="*80)

print(f"\nTRAIN DATA:")
print(f"  - Trước: {application_train.shape}")
print(f"  - Sau : {train_transformed.shape}")
print(f"  - Tăng: {train_transformed.shape[1] - application_train.shape[1]} features (+{((train_transformed.shape[1] - application_train.shape[1])/application_train.shape[1]*100):.1f}%)")

# print(f"\nTEST DATA:")
# print(f"  - Trước: {test_data.shape}")
# print(f"  - Sau : {test_transformed.shape}")
# print(f"  - Tăng: {test_transformed.shape[1] - test_data.shape[1]} features (+{((test_transformed.shape[1] - test_data.shape[1])/test_data.shape[1]*100):.1f}%)")

# Hiển thị một số features mới
print(f"\nMột số features mới được tạo:")
new_cols = [col for col in train_transformed.columns if col not in application_train.columns]
print(f"Total new features: {len(new_cols)}")
print(f"Examples: {new_cols[:10]}")

SO SÁNH TRƯỚC VÀ SAU FEATURE ENGINEERING

TRAIN DATA:
  - Trước: (307511, 122)
  - Sau : (307511, 557)
  - Tăng: 435 features (+356.6%)

Một số features mới được tạo:
Total new features: 471
Examples: ['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_ANNUITY_PERCENT', 'FAMILY_CNT_INCOME_PERCENT', 'CHILDREN_CNT_INCOME_PERCENT', 'CREDIT_TERM', 'BIRTH_EMPLOYED_PERCENT', 'EMPLOYED_REGISTRATION_PERCENT', 'BIRTH_REGISTRATION_PERCENT', 'ID_REGISTRATION_DIFF']


## Tổng Kết

### Phần 1: Áp dụng từng hàm
- Đã triển khai và demo các hàm feature engineering riêng lẻ
- Mỗi hàm được test để xem sự thay đổi của features

### Phần 2: Sklearn Pipeline
- Đã tạo complete pipeline với các custom transformers
- Pipeline có thể fit trên train data và transform cả train lẫn test data
- Pipeline có thể được lưu và tái sử dụng

### Ưu điểm của Pipeline:
1. **Tái sử dụng**: Fit một lần, transform nhiều lần
2. **Nhất quán**: Đảm bảo cùng một quy trình cho train và test
3. **Dễ bảo trì**: Code được tổ chức theo modules
4. **Production-ready**: Có thể deploy trực tiếp

### Các bước tiếp theo:
- Feature Selection (SelectKBest)
- Standardization (StandardScaler)  
- Model Training và Evaluation